### Importing libraries

In [1]:
import json
import pandas as pd
from uuid import uuid4
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Importing Slides JSON

In [453]:
# importing slides
with open('./slides-13.09.json') as t:
    data = json.load(t)

In [907]:
print('SLIDE STRUCTURE')
print(print(data['slides'][10]))
print()
print("========================================================")
print('PAGE ELEMENTS')
print(data['slides'][10]['pageElements'][0])
print()
print("========================================================")
print('TEXT CONTENT')
print(print(data['slides'][10]['pageElements'][0]['shape']['text']['textElements'][1]['textRun']['content']))

SLIDE STRUCTURE
{'objectId': 'gdb6001168f_0_1370', 'pageElements': [{'objectId': 'gdb6001168f_0_1371', 'size': {'width': {'magnitude': 3000000, 'unit': 'EMU'}, 'height': {'magnitude': 3000000, 'unit': 'EMU'}}, 'transform': {'scaleX': 0.7121, 'scaleY': 0.1013, 'translateX': 7007625, 'translateY': 1835125, 'unit': 'EMU'}, 'shape': {'shapeType': 'TEXT_BOX', 'text': {'textElements': [{'endIndex': 32, 'paragraphMarker': {'style': {'lineSpacing': 115, 'alignment': 'START', 'indentStart': {'unit': 'PT'}, 'indentEnd': {'unit': 'PT'}, 'spaceAbove': {'unit': 'PT'}, 'spaceBelow': {'unit': 'PT'}, 'indentFirstLine': {'unit': 'PT'}, 'direction': 'LEFT_TO_RIGHT', 'spacingMode': 'NEVER_COLLAPSE'}}}, {'endIndex': 32, 'textRun': {'content': 'Product Analyst AND Adventuress\n', 'style': {'backgroundColor': {}, 'foregroundColor': {'opaqueColor': {'themeColor': 'LIGHT1'}}, 'bold': False, 'italic': False, 'fontFamily': 'Poppins', 'fontSize': {'magnitude': 9, 'unit': 'PT'}, 'baselineOffset': 'NONE', 'smallCa

### Processing slides

In [454]:
#extract slides out of presentation
def slides_parser(raw_presentation):
    slides = pd.DataFrame(raw_presentation)
    return slides

In [455]:
temp_slides = slides_parser(data['slides'])
ids = []
for ind, i in enumerate(temp_slides['slideProperties']):
    try:
        if i['isSkipped'] == True:
            ids.append(ind)
    except:
        continue

slides = temp_slides.drop(ids)
print("No. of objects: " + "\033[1m"+str(slides.size))

No. of objects: 404


In [456]:
bio_objects = {}

### Extracting Text Elements

In [457]:
#extract text elements out slides
def text_element_parser(text_elements, page_element_no, slide_no):
    bio_objects[str(slide_no)][str(page_element_no)] = []
    for i in text_elements:
        try:
            bio_objects[str(slide_no)][str(page_element_no)].append(str(i['textRun']['content']))
        except:
            continue

def page_elements_parser(slide_page_elements, slide_no):
    bio_objects[str(slide_no)] = {}
    for idx, i in enumerate(slide_page_elements):
        try:
            text_element_parser(i['shape']['text']['textElements'], idx, slide_no)
        except:
            continue

In [458]:
def slides_parser(slides):    
    for ind in slides.index:
        try:
            page_elements_parser(slides['pageElements'][ind], ind)
        except:
            continue
slides_parser(slides)
print("\033[1m"+ str(len(bio_objects))+ " slides")

101 slides


### Removing non-Bio Slides

In [459]:
#remove non-Bio slides
def remove_non_bio_slides():
    for i in bio_objects.copy().keys():
        if len(bio_objects[i]) < 5 or i == '2':
            del bio_objects[i]
remove_non_bio_slides()
print("\033[1m"+ str(len(bio_objects))+ " slides")

90 slides


In [942]:
for i in bio_objects['11']:
    print(i+ ":")
    print(bio_objects['11'][i])
    print()

0:
['Product Developer AND Viking\n']

1:
['Tony is a passionate DevOps Engineer, dedicated to continuous learning with a keen interest in infrastructure, cloud and developer experience.\n']

2:
['Infrastructure, automation, pipelines\n']

3:
['Node JS, AWS,Terraform, Ansible, Kubernetes, Serverless, Agile, Stakeholder Management\n']

4:
['Openr\n', 'Azzurri Group\n', 'Belron\n', 'Capital One\n', 'Costa\n', 'Konsentus\n']

5:
['Hospitality\n', 'Banking\n', 'Fintech\n', '\n']

6:
['AWS\n', 'Terraform\n', 'Kubernetes\n', 'Ansible\n', 'Serverless\n', 'Javascript\n', '\n', '\n']

7:
['Tony Young \n']

9:
['Infrastructure & Backend Developer\n', 'Azzurri Group\n', 'A dynamic multi user application for ordering and payment in the casual dining sector, Tony helped by architecting the full application and creating all infrastructure in AWS using  Terraform as well as building out all the Node microservices.\n', '\n', 'Created multiple Node microservices\n', 'Built the full application with Ter

## Classification

### Names identifier

In [673]:
def name_identifier(bio_element):
    if len(bio_element) <= 4:
        for j in bio_element:
            if 'AND' not in j and 'OVERVIEW' not in j and len(j.split(' ')) < 5 and len(j.split('\n')) < 4 and not j.isspace()\
            and ((len(set(key_clients) & set(j.split('\n'))) == 0) and (len(set(key_techs) & set(j.split('\n'))) == 0) and (len(set(key_industries) & set(j.split('\n'))) == 0))\
            and ((len(set(key_clients) & set(j.split(' '))) == 0) and (len(set(key_techs) & set(j.split(' '))) == 0) and (len(set(key_industries) & set(j.split(' '))) == 0)) :
                return j.rstrip()
            else:
                continue
    else:
        return False

In [902]:
for i in bio_objects['11']:
    print(name_identifier(bio_objects['11'][i]))

None
None
None
None
False
None
False
Tony Young
False
False
False


### AND Titles identifier

In [903]:
def and_title_identifier(bio_element):
    if len(bio_element) <= 4:
        for j in bio_element:
#             print(j)
            if 'AND' in j and 'AND Digital' not in j and len(j.split(' ')) <= 15 and not j.isspace():
                return j[j.find('AND'):].replace('\n','')
            else:
                continue
    else:
        return False

In [905]:
slide_no = '11'
for i in bio_objects[slide_no]:
    print(and_title_identifier(bio_objects[slide_no][i]))

AND Viking
None
None
None
False
None
False
None
False
False
False


### Experiences Identifier

In [728]:
def experience_normalizer(experience):
    experience_object = {"role": "", "organisation":"", "description":""}
    experience = [xp for xp in experience if xp != '\n']
    for ind, i in enumerate(experience):
        if i == '\n' or i == '':
            continue
        else:
            if ind == 0:
                experience_object["role"] = i.rstrip()
            elif ind == 1:
                experience_object["organisation"] = i.rstrip()
            else:
                experience_object["description"] += i.rstrip() + " "
    return experience_object

def experiences_identifier(bio_element):
    if len("".join(bio_element).replace("\n","")) >= 215 and 'OVERVIEW' not in "".join(bio_element).replace("\n",""):
        return experience_normalizer(bio_element)
    else:
        return False
#     experiences = []
#     for ind, i in enumerate(bio_element):
#         try:
#             if len("".join(i).replace("\n","")) >= 175:
#                 exp_list = []
#                 for j in i:
#                     if j.strip() == '':
#                         continue
#                     else:
#                         exp_list.append(j.strip())
#                 experiences.append(experience_normalizer(exp_list))
#             else:
#                 continue
#         except:
#             continue
#     return experiences

In [906]:
# print(bio_objects['61'])
print()
for i in bio_objects['11']:
    print(experiences_identifier(bio_objects['11'][i]))


False
False
False
False
False
False
False
False
{'role': 'Infrastructure & Backend Developer', 'organisation': 'Azzurri Group', 'description': 'A dynamic multi user application for ordering and payment in the casual dining sector, Tony helped by architecting the full application and creating all infrastructure in AWS using  Terraform as well as building out all the Node microservices. Created multiple Node microservices Built the full application with Terraform as IaC Implemented Logzio for logging and monitoring '}
{'role': 'DevOps Engineer', 'organisation': 'Belron', 'description': 'Tony was tasked with building the infrastructure for a POC to consume historical data from Belron’s global business units and consolidate that data into one secure application Built out certificate based authentication across the full application from client to server Created full infrastructure across multiple environments Created CI/CD pipelines using a suite of AWS resources '}
{'role': 'Devops Engine

### Core_focus

### Key_Skills

### ML Classifier

In [252]:
train_data = {}
bio_objects_copy = bio_objects.copy()

In [253]:
#slide 4
train_data['role_and_title'] = bio_objects_copy['4']['0']
train_data['overview'] = bio_objects_copy['4']['1']
train_data['key_skills'] = bio_objects_copy['4']['2']
train_data['core_focus'] = bio_objects_copy['4']['3']
train_data['clients'] = bio_objects_copy['4']['4']
train_data['industries'] = bio_objects_copy['4']['5']
train_data['tech_stack'] = bio_objects_copy['4']['6']
train_data['name'] = bio_objects_copy['4']['7']
train_data['experiences'] = [bio_objects_copy['4']['9']]
train_data['experiences'].append(bio_objects_copy['4']['10'])
train_data['experiences'].append(bio_objects_copy['4']['11'])

#slide 11
train_data['role_and_title'].append(bio_objects_copy['11']['0'])
train_data['overview'].append(bio_objects_copy['11']['1'])
train_data['core_focus'].append(bio_objects_copy['11']['2'])
train_data['key_skills'].append(bio_objects_copy['11']['3'])
train_data['clients'].append(bio_objects_copy['11']['4'])
train_data['industries'].append(bio_objects_copy['11']['5'])
train_data['tech_stack'].append(bio_objects_copy['11']['6'])
train_data['name'].append(bio_objects_copy['11']['7'])
train_data['experiences'].append([bio_objects_copy['11']['9']])
train_data['experiences'].append(bio_objects_copy['11']['10'])
train_data['experiences'].append(bio_objects_copy['11']['11'])

#slide 16
train_data['role_and_title'].append(bio_objects_copy['16']['0'])
train_data['overview'].append(bio_objects_copy['16']['1'])
train_data['core_focus'].append(bio_objects_copy['16']['2'])
train_data['key_skills'].append(bio_objects_copy['16']['3'])
train_data['clients'].append(bio_objects_copy['16']['4'])
train_data['industries'].append(bio_objects_copy['16']['5'])
train_data['tech_stack'].append(bio_objects_copy['16']['6'])
train_data['name'].append(bio_objects_copy['16']['7'])
train_data['experiences'].append([bio_objects_copy['16']['8']])
train_data['experiences'].append(bio_objects_copy['16']['9'])
train_data['experiences'].append(bio_objects_copy['16']['10'])

#slide 28
train_data['role_and_title'].append(bio_objects_copy['28']['8'])
train_data['overview'].append(bio_objects_copy['28']['4'])
train_data['key_skills'].append(bio_objects_copy['28']['6'])
train_data['core_focus'].append(bio_objects_copy['28']['5'])
train_data['clients'].append(bio_objects_copy['28']['12'])
train_data['industries'].append(bio_objects_copy['28']['11'])
train_data['tech_stack'].append(bio_objects_copy['28']['7'])
train_data['name'].append(bio_objects_copy['28']['3'])
train_data['experiences'].append([bio_objects_copy['28']['0']])
train_data['experiences'].append(bio_objects_copy['28']['1'])
train_data['experiences'].append(bio_objects_copy['28']['2'])

#slide 29
train_data['role_and_title'].append(bio_objects_copy['29']['9'])
train_data['overview'].append(bio_objects_copy['29']['7'])
train_data['key_skills'].append(bio_objects_copy['29']['10'])
train_data['core_focus'].append(bio_objects_copy['29']['11'])
train_data['clients'].append(bio_objects_copy['29']['0'])
train_data['industries'].append(bio_objects_copy['29']['1'])
train_data['tech_stack'].append(bio_objects_copy['29']['2'])
train_data['name'].append(bio_objects_copy['29']['8'])
train_data['experiences'].append([bio_objects_copy['29']['4']])
train_data['experiences'].append(bio_objects_copy['29']['5'])
train_data['experiences'].append(bio_objects_copy['29']['13'])

#slide 33
train_data['role_and_title'].append(bio_objects_copy['33']['10'])
train_data['overview'].append(bio_objects_copy['33']['6'])
train_data['key_skills'].append(bio_objects_copy['33']['8'])
train_data['core_focus'].append(bio_objects_copy['33']['7'])
train_data['clients'].append(bio_objects_copy['33']['5'])
train_data['industries'].append(bio_objects_copy['33']['4'])
train_data['tech_stack'].append(bio_objects_copy['33']['3'])
train_data['name'].append(bio_objects_copy['33']['11'])
train_data['experiences'].append([bio_objects_copy['33']['0']])
train_data['experiences'].append(bio_objects_copy['33']['1'])
train_data['experiences'].append(bio_objects_copy['33']['2'])

#slide 40
train_data['role_and_title'].append(bio_objects_copy['40']['0'])
train_data['overview'].append(bio_objects_copy['40']['1'])
train_data['key_skills'].append(bio_objects_copy['40']['3'])
train_data['core_focus'].append(bio_objects_copy['40']['2'])
train_data['clients'].append(bio_objects_copy['40']['4'])
train_data['industries'].append(bio_objects_copy['40']['5'])
train_data['tech_stack'].append(bio_objects_copy['40']['6'])
train_data['name'].append(bio_objects_copy['40']['7'])
train_data['experiences'].append([bio_objects_copy['40']['9']])
train_data['experiences'].append(bio_objects_copy['40']['10'])
train_data['experiences'].append(bio_objects_copy['40']['11'])

#slide 65
train_data['role_and_title'].append(bio_objects_copy['65']['0'])
train_data['overview'].append(bio_objects_copy['65']['1'])
train_data['key_skills'].append(bio_objects_copy['65']['3'])
train_data['core_focus'].append(bio_objects_copy['65']['2'])
train_data['clients'].append(bio_objects_copy['65']['4'])
train_data['industries'].append(bio_objects_copy['65']['5'])
train_data['tech_stack'].append(bio_objects_copy['65']['6'])
train_data['name'].append(bio_objects_copy['65']['7'])
train_data['experiences'].append([bio_objects_copy['65']['9']])
train_data['experiences'].append(bio_objects_copy['65']['10'])
train_data['experiences'].append(bio_objects_copy['65']['11'])

#slide 66
train_data['role_and_title'].append(bio_objects_copy['66']['10'])
train_data['overview'].append(bio_objects_copy['66']['11'])
train_data['key_skills'].append(bio_objects_copy['66']['13'])
train_data['core_focus'].append(bio_objects_copy['66']['12'])
train_data['clients'].append(bio_objects_copy['66']['6'])
train_data['industries'].append(bio_objects_copy['66']['7'])
train_data['tech_stack'].append(bio_objects_copy['66']['8'])
train_data['name'].append(bio_objects_copy['66']['9'])
train_data['experiences'].append([bio_objects_copy['66']['3']])
train_data['experiences'].append(bio_objects_copy['66']['4'])
train_data['experiences'].append(bio_objects_copy['66']['5'])

#slide 72
train_data['role_and_title'].append(bio_objects_copy['72']['0'])
train_data['overview'].append(bio_objects_copy['72']['1'])
train_data['key_skills'].append(bio_objects_copy['72']['3'])
train_data['core_focus'].append(bio_objects_copy['72']['2'])
train_data['clients'].append(bio_objects_copy['72']['4'])
train_data['industries'].append(bio_objects_copy['72']['5'])
train_data['tech_stack'].append(bio_objects_copy['72']['6'])
train_data['name'].append(bio_objects_copy['72']['7'])
train_data['experiences'].append([bio_objects_copy['72']['9']])
train_data['experiences'].append(bio_objects_copy['72']['10'])
train_data['experiences'].append(bio_objects_copy['72']['11'])
train_data['experiences'].append(bio_objects_copy['72']['12'])

#slide 84
train_data['role_and_title'].append(bio_objects_copy['84']['3'])
train_data['overview'].append(bio_objects_copy['84']['4'])
train_data['key_skills'].append(bio_objects_copy['84']['6'])
train_data['core_focus'].append(bio_objects_copy['84']['5'])
train_data['clients'].append(bio_objects_copy['84']['7'])
train_data['industries'].append(bio_objects_copy['84']['8'])
train_data['tech_stack'].append(bio_objects_copy['84']['9'])
train_data['name'].append(bio_objects_copy['84']['10'])
train_data['experiences'].append([bio_objects_copy['84']['0']])
train_data['experiences'].append(bio_objects_copy['84']['1'])
train_data['experiences'].append(bio_objects_copy['84']['2'])

#slide 90
train_data['role_and_title'].append(bio_objects_copy['90']['10'])
train_data['overview'].append(bio_objects_copy['90']['11'])
train_data['key_skills'].append(bio_objects_copy['90']['13'])
train_data['core_focus'].append(bio_objects_copy['90']['12'])
train_data['clients'].append(bio_objects_copy['90']['6'])
train_data['industries'].append(bio_objects_copy['90']['7'])
train_data['tech_stack'].append(bio_objects_copy['90']['8'])
train_data['name'].append(bio_objects_copy['90']['9'])
train_data['experiences'].append([bio_objects_copy['90']['3']])
train_data['experiences'].append(bio_objects_copy['90']['4'])
train_data['experiences'].append(bio_objects_copy['90']['5'])

#slide 91
train_data['role_and_title'].append(bio_objects_copy['91']['5'])
train_data['overview'].append(bio_objects_copy['91']['1'])
train_data['key_skills'].append(bio_objects_copy['91']['3'])
train_data['core_focus'].append(bio_objects_copy['91']['2'])
train_data['clients'].append(bio_objects_copy['91']['6'])
train_data['industries'].append(bio_objects_copy['91']['7'])
train_data['tech_stack'].append(bio_objects_copy['91']['8'])
train_data['name'].append(bio_objects_copy['91']['0'])
train_data['experiences'].append([bio_objects_copy['91']['9']])
train_data['experiences'].append(bio_objects_copy['91']['10'])
train_data['experiences'].append(bio_objects_copy['91']['11'])

In [943]:
train_data['tech_stack']

['scrum @ scale certified practitioner',
 'professional scrum master i certified',
 'jira',
 'aws',
 'terraform',
 'kubernetes',
 'ansible',
 'serverless',
 'javascript',
 'jira',
 'azure devops',
 'scrum',
 'justinmind, invision, figma, miro',
 'salesforce',
 'power bi',
 'figma',
 'miro / mural',
 'pen & paper',
 'post-its',
 'javascript',
 'react',
 'react native',
 'typescript',
 'node.js',
 'tdd, jest',
 'figma',
 'miro',
 'jira',
 'trello',
 'quantum metrics',
 'log rocket',
 'node.js, typescript',
 'reactjs, gatsbyjs',
 'java',
 'kafka',
 'graphql, rest, api design',
 'sql, nosql',
 'docker, circleci, jenkins',
 'javascript',
 'react',
 'react native',
 'node',
 'docker',
 'aws',
 'contentful',
 'gatsbyjs',
 'tools',
 'figma',
 'figjam',
 'miro',
 'keynote',
 'react / redux / thunk',
 'vanilla js, typescript',
 'jest, enzyme',
 'node express',
 'scss, jss, less, css',
 'graphql',
 'exposure to next.js',
 'certified contentful cms',
 'angular',
 'aws',
 'nodejs',
 'serverless',
 

In [472]:
key_clients = ['Squad', 'Whitbread', 'London\n', 'Lloyds of London','Department', 'Azzurri\n', 'Azzurri', 'Azzurri Group', 'Azzurri Group\n', 'Loyds', 'Walters\n', 'Openr\n', 'Openr', 'OpenR', ' Oxford','Oxford','Aviva', 'Group', 'Home Office', 'Citi', 'S&P', 'AXA', 'TUI', 'Mopro', 'FWD', 'FinTech/InsurTech', 'e-Commerce', 'Ineos', 'Invesco', 'TFL', 'Taylor', 'Royal', 'Shawbrook', 'ByBits', 'Berry', 'Mporium', 'Lego', 'Nanopore', 'WorldRemit', 'WorldRemit', 'Lenio', 'Splink', 'HSSMI', 'Ironfly', 'Ford', 'Vida', 'MHHS', 'Jet', 'Cancer', 'Bharat', 'Idea', 'Audi', 'Oxford Nanopore', 'Zenith', 'Zenith', 'British', 'United', 'IAG', 'Wine', 'Dining', 'Solotel', 'Say', 'MOD', 'Bayswater', 'BNP', 'Honda', 'Rakuten', 'Automobile', 'RSA', 'BAE', 'Robert Walters  \n', 'Walters', 'Robert', 'Titan', 'Digital']

key_techs = ['Delivery', 'Product','Product Delivery','Agile/Scrum','AWS', 'Tech', 'Competitor Analysis', 'Analysis', 'Scrum', 'Agile', 'Figma', 'Full', 'Full-Stack', 'Jira\n', 'Jira', 'Front', 'Front-End', 'Front-end', 'Frontend', 'Back', 'Back-end', 'Back-End', 'Backend', 'Engineer', 'Product', 'Engineer', 'End', 'Node.js\n', 'Postgres\n','Infrastructure', 'Experience', 'Strategic', 'UX', 'UI', 'Design', 'Figma', 'Finance', 'User', 'CMS', 'Marketing', 'Mobile', 'e-commerce', 'E-Commerce', 'E-commerce', 'Business', 'Address', 'Language', 'SQL', 'DevOps', 'Customer', 'NodeJS', 'INDUSTRY', 'Loyalty', 'KEY', 'B2B', 'Brand', 'CRM', 'Typescript', 'Kafka', 'Tester', 'eCommerce',  ' | Coaching', 'DevOps/SD/Product', 'Software', 'Web', 'Infrastructure,', 'NA', 'Automated', 'Machine', 'Node.js', 'Postgres', 'Card', 'Systems', 'Hi-Fi', 'design', 'Design', 'Associate', 'Product Analysis & Delivery', 'Digital', 'ideas', 'products', ' Product Developer AND Sudoku Solver']

key_industries = ['Events\n', 'Hospitality\n', 'Hospitality','Music\n', 'B2C\n', 'Retail\n', 'Retail', 'Casual', 'Telecommunications', 'Gambling', 'Gmabling\n', 'Travel', 'Public', 'UK', 'FinTech', 'Media', 'Insurance', 'Defence', 'Banking', 'Health', 'Education', 'Healthcare', 'Real', 'Professional','Hotels', 'Insurtech', 'Games', 'International', 'Recruitment', 'Fintech', 'Financial', 'Pharmaceutical', 'Charity', 'Railway', 'Energy', 'Manufacturing', 'Oil', 'Aviation', 'Leisure', 'Aerospace', 'Military', 'Advertisement', 'Music']

In [291]:
# train_data['clients'].append(key_clients)
# train_data['tech_stack'].append(key_techs)
# train_data['industries'].append(key_industries)

In [254]:
categories = ['role_and_title',
 'overview',
 'key_skills',
 'core_focus',
 'clients',
 'industries',
 'tech_stack',
 'name',
 'experiences']

train_data_copy = {}

for i in categories:
    train_data_copy[i] = []

train_data_copy

{'role_and_title': [],
 'overview': [],
 'key_skills': [],
 'core_focus': [],
 'clients': [],
 'industries': [],
 'tech_stack': [],
 'name': [],
 'experiences': []}

In [255]:
def list_cleaner(key):
    for i in train_data[key]:
        if type(i) == list:
            try:
                for n in i:
                    if type(n) == list:
                        for j in n:
                            if len(j) > 1 and not (j.strip() == ''):
                                train_data_copy[key].append(j.strip().lower())                            
                    elif len(n) > 1 and not (n.strip() == ''):
                        train_data_copy[key].append(n.strip().lower())
            except:
                continue
        else:
            if len(i) > 1 and not (i.strip() == ''):
                train_data_copy[key].append(i.strip().lower())
#     return train_data_copy[key]

In [256]:
for i in categories:
    list_cleaner(i)

In [257]:
train_data = train_data_copy

### Learning algorithms

In [258]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

#### Combine text data and labels

In [259]:
del train_data['experiences']
del train_data['name']
del train_data['core_focus']
del train_data['role_and_title']
del train_data['key_skills']
# del train_data['overview']

In [946]:
X = []
y = []

for category, texts in train_data.items():
    X.extend(texts)
    y.extend([category] * len(texts))

print(X[20:36])
print()
print(y[20:36])

['-', 'overview', 'sajidah joined and digital in 2022 bringing in over 1 years experience as a front-end developer.', 'tony is a passionate devops engineer, dedicated to continuous learning with a keen interest in infrastructure, cloud and developer experience.', 'anshu is a product owner having 10+ years it industry experience with variety of domains and clients', 'jessica joined and digital in 2021, after graduating from an ma in design innovation management and interning in various design roles.', 'julia joined and digital in 2022 as a product developer with strong skills in frontend and', 'an interest in backend technologies.', 'raihan joined and digital in may 2022, bringing with him 2 years product management experience.', 'experienced full-stack developer', 'with 8 years of experience', 'spanning', 'back-end,', 'front-end and devops', 'aviraj joined and digital after graduating with compsci yii and has since been put in pivotal roles on various projects and clients', 'overview']

### Split data into training and testing sets

In [261]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    random_state=42)

### Vectorize text using TF-IDF

In [262]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [263]:
print(X_train_tfidf[20:26])

  (0, 181)	0.5834546404357493
  (0, 287)	0.5834546404357493
  (0, 393)	0.5649436831295497
  (1, 88)	1.0
  (2, 200)	0.6764043818115826
  (2, 353)	0.7365304557627546
  (3, 405)	1.0
  (4, 319)	1.0
  (5, 149)	1.0


### Training a Multinomial Naive Bayes classifier

In [264]:
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

### Make predictions & Model Evaluation

In [265]:
y_pred = classifier.predict(X_test_tfidf)

In [266]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.77
Classification Report:
              precision    recall  f1-score   support

     clients       0.89      0.80      0.84        20
  industries       0.78      0.70      0.74        20
    overview       1.00      0.60      0.75         5
  tech_stack       0.64      0.84      0.73        19

    accuracy                           0.77        64
   macro avg       0.83      0.74      0.76        64
weighted avg       0.79      0.77      0.77        64



In [965]:
sample_data = ['Hospitality']
    
sample_data_prediction = classifier.predict(vectorizer.transform(sample_data))

In [966]:
for text, predicted_label in zip([sample_data], sample_data_prediction):
    print("Text: " + str(text[0]))
    print("Predicted Label: " +str(predicted_label))
    print()

Text: Hospitality
Predicted Label: industries



In [597]:
def sub_sections_classifier(elements):
    '''
Main function for predicting slides elements
    '''
    vectorized_elements = []
    for elem in elements:
        vectorized_elements.append(elem.lower().rstrip())
    vectorized_elements = vectorizer.transform(elements)
    return classifier.predict(vectorized_elements)

In [596]:
def sub_sections_evaluater(sub_section):
    '''
Function to evaluate what is the category of a slide subsection
    '''
    labels = sub_sections_classifier(sub_section)
    return np.unique(labels)[0]
    

In [760]:
def overview_evaluater(element):
    words = [elem.split(' ') for elem in element]
    if len(words) < 2:
        words = [x for x in words.copy()[0]]
    if len(words) > 8:
        overview_bag_of_words = ['joined','yrs','years','experience','experienced', 'passionate', 'background', 'OVERVIEW\n']
        return True in [word in overview_bag_of_words for word in words]
    else:
        return False

In [767]:
# print(bio_objects['61'])
# print(overview_evaluater(bio_objects['18']['1']))
print(overview_evaluater(bio_objects['61']['7']))
print()
print(sub_sections_evaluater(bio_objects['61']['7']))
print()
print(element_evaluater(bio_objects['61']['7']))
# for i in bio_objects['66']:
#     print(bio_objects['66'][i])
#     print(element_evaluater(bio_objects['66'][i]))
#     print()
# print("".join(bio_objects['66']['11']).rstrip())

False

overview

{'overview': 'Caio joined AND Digital in 2022 as a Product Developer with strong skills in Backend (NodeJS) and commercial experience with  database and BI technologies.'}


In [742]:
def element_evaluater(element):
    if name_identifier(element) not in [False, None, '-', '']:
        return {"name": name_identifier(element)}
    elif and_title_identifier(element) not in [False, None, '-', '']:
        return {"and_title": and_title_identifier(element)}
    elif sub_sections_evaluater(element) == 'overview' or overview_evaluater(element):
        if 'OVERVIEW\n' in element or 'OVERVIEW' in element:
            try:
                element.remove('OVERVIEW')
                element.remove('OVERVIEW\n')
                element.remove('\n')
            except:
                return {"overview": "".join(element).rstrip()}                    
            return {"overview": "".join(element).rstrip()}
        else:
            return {"overview": "".join(element).rstrip()}
    elif experiences_identifier(element) not in [False, None, '-', '']:
        return {"experience": experiences_identifier(element)}
    else:
            return

## Data Processor

In [743]:
bio_objects = {}
slides_parser(slides)
remove_non_bio_slides()

In [776]:
def slides_processor(slides):
    processed_slides = []
    for slide in slides:
        temp_id = str(uuid4())
        name = ''
        and_title = ''
        experiences = []
        overview = ''

        for element in slides[slide]:
            classified_element =  element_evaluater(slides[slide][element])
            if classified_element == None:
                continue
            elif 'name' in classified_element.keys() and name == '':
                name = classified_element['name']
            elif 'and_title' in classified_element.keys() and and_title == '':
                and_title = classified_element['and_title']
            elif 'overview' in classified_element.keys() and overview == '':
                overview = classified_element['overview']
            elif 'experience' in classified_element.keys():
                experiences.append(classified_element['experience'])            
            else:
                continue          
        
        processed_slides.append({
            "slide_no": str(slide),
            "key": 'BIO#'+ str(uuid4()),
            "name": name,
            "firstName": name.split(' ')[0],
            "lastName": name.split(' ')[-1],
            "andTitle": and_title,
            "experiences": experiences,
            "overview": overview
        })
    return processed_slides
    

Construir objeto parcialmente <br>
1 - Fazer nomes <br>
2 - Fazer AND Titles <br>
3 - Fazer Experiencias <br>

In [974]:
df = pd.DataFrame(slides_processor(bio_objects))
# df.iloc[25:55]
# df.head(25)
# df.tail(25)
# print(list(df[df['firstName'] == 'Musab']['overview']))
# df[df['overview'] == '']

['AWS Certified developer with professional experience with serverless computing. Worked in Travel and Tourism industry']


In [938]:
# df.iloc[37:51,2:]
df.head(30)

,slide_no,key,name,firstName,lastName,andTitle,experiences,overview
0,4,BIO#c09e0905-483d-4831-934a-072671da1ee7,Alex Minssieux,Alex,Minssieux,AND Board Game Geek,"[{'role': 'Agile Coach and Scrum Master', 'org...",Technical delivery lead and Agile coach with o...
1,5,BIO#980f6744-18df-42b7-9257-be1c5a80ebc0,Alex Minssieux,Alex,Minssieux,AND Board Game Geek,"[{'role': 'Agile Coach and Scrum Master', 'org...",Technical delivery lead and Agile coach with o...
2,6,BIO#9858b224-c07a-43c7-835d-98da69733140,Ayodele Alakija,Ayodele,Alakija,AND Adventure Animal,"[{'role': 'Frontend Developer', 'organisation'...",Ayodele joined AND Digital as a frontend devel...
3,7,BIO#d127fbfc-4ffc-4e12-9080-c0a6e54bacbd,Barney Loosemore,Barney,Loosemore,AND Avid Climber,"[{'role': 'Software Developer', 'organisation'...",Barney joined AND Digital in 2022 bringing in ...
4,8,BIO#5bd999d2-f276-4b94-9656-aa53a103f805,Hailey Pun,Hailey,Pun,AND Film Aficionado,"[{'role': 'Frontend Team Lead', 'organisation'...","With over 3 years of development experience, H..."
5,9,BIO#9094be11-cb59-442b-a2b9-5c5b4fd2548a,Peter Holdsworth,Peter,Holdsworth,AND Maths Geek,"[{'role': 'Senior Product Developer', 'organis...",Peter is a senior product developer with over ...
6,11,BIO#df623465-4d93-4431-8bab-37eccc0f56f9,Tony Young,Tony,Young,AND Viking,[{'role': 'Infrastructure & Backend Developer'...,"Tony is a passionate DevOps Engineer, dedicate..."
7,13,BIO#673ac1b4-9dd8-40e8-836a-4b29bfd24a71,Jakob Ritt,Jakob,Ritt,AND Doodler,"[{'role': 'Experience Designer', 'organisation...",-\nOVERVIEW\n\nSystematic design thinker and c...
8,14,BIO#b4c387f0-7f15-460e-8029-30f0077be336,Christine Efedua,Christine,Efedua,,"[{'role': 'AWS Cloud Practitioner', 'organisat...",Christine joined AND in 2021 as an Associate C...
9,15,BIO#9ebf17e0-fbb7-44bd-943b-430f8d938b02,Jamie Greenwood,Jamie,Greenwood,AND Golf Nut,"[{'role': 'Senior UI Designer', 'organisation'...",6+ years experience as a Digital Designer with...


In [986]:
df.to_csv('./bios.csv')

In [975]:
for i in list(df[df['firstName'] == 'Ashraf']['experiences']):
    print(i)
#     for j in i:
#         print(j['organisation'])

[{'role': 'Front End Developer', 'organisation': 'Premier Inn (Whitbread plc)', 'description': 'Ashraf has been involved in the Business Booker team, helping create a greenfield project, collaborating in cross teams. He has helped introduce best practices and been involved in integrating CI/CD pipelines, retrieving the quality scores of web pages. Moreover, he has integrated E2E testing for the journey of the application. '}, {'role': 'Full Stack Developer', 'organisation': 'Luna (Internal project)', 'description': 'Ashraf has worked on a complex internal project and developed features which were specific to user roles and company data , including utilising both AWS lambdas and database migrations. Additionally, Ashraf helped increase the type safety of the codebase, thus allowing errors to be spotted earlier. He also helped onboard new members to the team and assisted more junior members of the team in completing tickets. '}, {'role': 'Front End Developer', 'organisation': 'Openr (Azz

In [63]:
processed_slides = []
            
for i in bio_objects.keys():
    temp_id = str(uuid4())
    name = name_identifier(bio_objects[i]).rstrip().split('\n')[0].split(' ')
    full_name = name_identifier(bio_objects[i]).rstrip()
    and_title = and_title_identifier(bio_objects[i])
    experiences = experiences_identifier(bio_objects[i])
    tech_stacks = []
    industries = []
    clients = []
    
    for elements in bio_objects[i]:
        if sub_sections_evaluater(bio_objects[i][elements]) == 'tech_stack':
            tech_stacks = bio_objects[i][elements]
        elif sub_sections_evaluater(bio_objects[i][elements]) == 'industries':
            industries = [x.rstrip() for x in bio_objects[i][elements]]
        elif sub_sections_evaluater(bio_objects[i][elements]) == 'clients':
            clients = bio_objects[i][elements]
    processed_slides.append({\
                  'key': 'USER#'+ temp_id,
                  'id': 'BIO#' + temp_id,
                  'full_name': full_name,
                  'firstName': name[0],
                  "lastName": name[-1],
                  'and_tile': and_title,
                  'experiences': experiences,
                  'tech_stack': tech_stacks,
                  'industries': industries,
                  'clients': clients                            
                 })

In [940]:
### processed_slides = pd.DataFrame(processed_slides)
print([x for x in processed_slides[processed_slides['firstName'] == 'Ashraf']['experiences']])

[[{'role': 'Front End Developer', 'organisation': 'Premier Inn (Whitbread plc)', 'description': 'Ashraf has been involved in the Business Booker team, helping create a greenfield project, collaborating in cross teams. He has helped introduce best practices and been involved in integrating CI/CD pipelines, retrieving the quality scores of web pages. Moreover, he has integrated E2E testing for the journey of the application. '}, {'role': 'Full Stack Developer', 'organisation': 'Luna (Internal project)', 'description': 'Ashraf has worked on a complex internal project and developed features which were specific to user roles and company data , including utilising both AWS lambdas and database migrations. Additionally, Ashraf helped increase the type safety of the codebase, thus allowing errors to be spotted earlier. He also helped onboard new members to the team and assisted more junior members of the team in completing tickets. '}, {'role': 'Front End Developer', 'organisation': 'Openr (Az

In [78]:
print([i for i in processed_slides[0:3]['industries']])

[['Academic Publishing', 'B2B and B2C', 'Financial Services', 'Food and Drink', 'Hospitality', 'Real Estate', 'Retail', 'Travel'], ['Academic Publishing', 'B2B and B2C', 'Financial Services', 'Food and Drink', 'Hospitality', 'Real Estate', 'Retail', 'Travel'], ['Hospitality', 'E-Commerce', 'Retail', 'Public Sector', '']]


In [ ]:
users = processed_slides.drop_duplicates(subset=['full_name']).iloc[:,0:3]
users.drop(columns=['id']).head(20)

In [ ]:
processed_slides = pd.DataFrame(processed_slides)
n_i = 80
print(list(processed_slides[n_i:n_i+1]['experiences'])[0][2])
print()
print(list(processed_slides[n_i:n_i+1]['name'])[0])

In [ ]:
char_count = []
for index, row in processed_slides.iterrows():
    for i in row['experiences']:
        if len(i['description']) > 0:
#         print(i['description'])
#         print()
            char_count.append(len(i['description']))
        else:
            continue
print(np.max(char_count))
print(np.average(char_count))
# print()
# print(char_count)

In [ ]:
bio_objects['61']['7']

In [851]:
my_json = [
    {
       "role": "Front End Developer",
       "organisation": "Premier Inn (Whitbread plc)",
       "description": "Ashraf has been involved in the Business Booker team, helping create a greenfield project, collaborating in cross teams. He has helped introduce best practices and been involved in integrating CI/CD pipelines, retrieving the quality scores of web pages. Moreover, he has integrated E2E testing for the journey of the application. "
    },
    {
       "role": "Full Stack Developer",
       "organisation": "Luna (Internal project)",
       "description": "Ashraf has worked on a complex internal project and developed features which were specific to user roles and company data , including utilising both AWS lambdas and database migrations. Additionally, Ashraf helped increase the type safety of the codebase, thus allowing errors to be spotted earlier. He also helped onboard new members to the team and assisted more junior members of the team in completing tickets. "
    },
    {
       "role": "Front End Developer",
       "organisation": "Openr (Azzurri Group)",
       "description": "Ashraf has built user interfaces and features which has solved the needs of restaurant chains. He has further helped developed a greenfield application for stakeholders to update the availabilities of their products, using best and modern front-end web practices. He has also helped increase the code end-to-end coverage of the applications, leading to them being more reliable and robust. "
    }
 ]

In [871]:
max_num_of_char = 0
num_of_char = []
for i in df['experiences']:
    for xp in i:
        num_of_char.append(len(xp['description']))
        if len(xp['role']) > max_num_of_char:
            max_num_of_char = len(xp['description'])
        else:
            continue

print('Max. number of characters in Experiences Description is: ' + str(max_num_of_char))
print()
print('Average number of Experience Description is ' + str(np.round(np.sum(num_of_char)/len(num_of_char),0)))
        

Max. number of characters in Experiences Description is: 308

Average number of Experience Description is 374.0
